In [ ]:
#default_exp per_patient

# Per-Patient

> Create a FHIR bundle of medications for a single patient.

As the first step in converting FHIR resources to the CDISC "Concomitant/Prior Medications" CM domain, we'll create a `Bundle` containing one `Patient` and any number of `MedicationAdministration`, `MedicationDispense`, `MedicationRequest` and `MedicationStatement` resources.

So that subsequent use of this bundle doesn't have to make any FHIR server requests, the bundle will also contain;
- `Medication` referenced by `medicationReference`
- `Condition` and/or `Observation` referenced by `reasonReference`

## See also

[vulcan_medication_bundle_getting_started.ipynb](https://colab.research.google.com/github/pete88b/smart-on-fhir-client-py-demo/blob/main/vulcan_medication_bundle_getting_started.ipynb) explains;
- Why we are not using `List` and
- Why we are reading FHIR resources as raw JSON

## TODO: Remove non-concomitant medications from the list

Identifying concomitant medications might get quite complicated - I'm assuming we won't be able to cover all logic needed when pulling data from the FHIR servers. I think it makes sense to pull all medications, then add a concomitant medication filter as a subsequent step.

### How are we defining concomitant medications? 

Any medication  
- that is not the medication being investigated
- that is being taken while a patient is participating in a study

We might also want to list subset of concomitant medications - i.e. thoes listed in exclusion criteria, relevant medications that the study would like to follow (e.g. concomitant use of ACE inhibitors might be important but single dose paracetamil might not).

To know if the medication was being taken while the patient was/is participating in a study, we could

could compare study participation
- study participation from `ResearchSubject.period`
- study duration from `ResearchStudy.period`
    - if either start or end date are missing from `ResearchSubject.period`
- user specified start and end date
    - if `ResearchStudy` etc are not in FHIR?
    
with start and end time of medication "administration"
- `MedicationStatement.effectiveX`, `MedicationStatement.dateAsserted`, `MedicationStatement.dosage`
    - don't forget `MedicationX.status` not-taken etc
- `MedicationRequest.authoredOn`, `MedicationRequest.encounter`, `MedicationRequest.dosageInstruction`, `MedicationRequest.basedOn`, `MedicationRequest.dispenseRequest` ...
    - Don't forget `MedicationRequest.doNotPerform`
- `MedicationDispense.daysSupply`, `MedicationDispense.whenPrepared`, `MedicationDispense.whenHandedOver`, `MedicationDispense.dosageInstruction`, `MedicationDispense.partOf`, `MedicationDispense.authorizingPrescription` 
- `MedicationAdministration.effectiveX`, partOf, supportingInformation ...


## TODO: think about a "human in the loop" to help with things &uarr; that will be hard to reliably automate


## Next steps

Might we want to
- define some kind of order of entries in the bundle
- think about how we handle resources that fail validation 
    - We can use https://inferno.healthit.gov/validator/ to validate the bundes created
        - TODO: Can we discuss how we want to action this output?

## Required resources?

The CM tab (https://wiki.cdisc.org/display/FHIR2CDISCUG/FHIR+to+CDISC+Mapping+User+Guide+Home FHIR-to-CDISC Mappings xlsx) lists the following resources;

- ~~`ResearchSubject` with `ResearchStudy`~~ NOT YET?
- `Subject`
- `MedicationStatement`
- `MedicationRequest`
- `MedicationDispense`
- `MedicationAdministration`
- ~~`Immunization`~~ don't think we're doing immunization yet?
- `Medication` referenced by `medicationReference`
- `Condition` or `Observation` referenced by `reasonReference`

TODO: For now, I'm just pulling the resources that Jay highlighted as required - we can easily add the others (o:

In [ ]:
#export
from vulcan_medication_bundle.core import *
from pathlib import Path
import json

In [ ]:
from io import StringIO
import requests

In [ ]:
#demo
import pandas as pd

In [ ]:
# api_base = 'http://hapi.fhir.org/baseR4'
api_base, patient_id = 'https://r4.smarthealthit.org', '11f2b925-43b2-45e4-ac34-7811a9eb9c1b'

In [ ]:
bundle = get_as_raw_json(api_base, 'MedicationRequest', dict(subject=patient_id))
print('Patient', patient_id, 'has', len(bundle['entry']), 'MedicationRequest resources')

GET https://r4.smarthealthit.org/MedicationRequest?_format=json&subject=11f2b925-43b2-45e4-ac34-7811a9eb9c1b
Patient 11f2b925-43b2-45e4-ac34-7811a9eb9c1b has 10 MedicationRequest resources


In [ ]:
# uncomment and run this cell to see the bundle as raw JSON
# bundle

## Create and save a single patient medication bundle

TODO: extract ref for `statusReasonReference` - see https://www.hl7.org/fhir/medicationdispense.html? maybe

In [ ]:
#export
def create_single_patient_medication_bundle(api_base, patient_id):
    "Return a Bundle containing one Patient and any number of MedicationX resources"
    result = new_bundle()
    references = []
    for resource_type, url_suffix in [
            ['Patient', dict(_id=patient_id)],
            ['MedicationRequest', dict(subject=f'Patient/{patient_id}')],
            ['MedicationDispense', dict(subject=f'Patient/{patient_id}')],
            ['MedicationAdministration', dict(subject=f'Patient/{patient_id}')],
            ['MedicationStatement', dict(subject=f'Patient/{patient_id}')]]:
        try:
            single_resource_bundle = get_as_raw_json(api_base, resource_type, url_suffix)
            while single_resource_bundle is not None and single_resource_bundle['total'] > 0:
                result['entry'].extend(single_resource_bundle['entry'])
                # TODO: xxx medicationReference and reasonReference might not be enough
                references.extend(extract_references(single_resource_bundle, ['medicationReference', 'reasonReference']))
                single_resource_bundle = get_next_as_raw_json(single_resource_bundle)
        except Exception as ex:
            print(f'Failed to get {resource_type}, {url_suffix} from {api_base}\n{ex}')
    for reference in set(references):
        try:
            result['entry'].extend(get_by_reference(api_base, reference))
        except Exception as ex:
            print(f'Failed to reference {reference} from {api_base}\n{ex}')
    return result

In [ ]:
bundle = create_single_patient_medication_bundle(api_base, patient_id)
bundle 

GET https://r4.smarthealthit.org/Patient?_format=json&_id=11f2b925-43b2-45e4-ac34-7811a9eb9c1b
GET https://r4.smarthealthit.org/MedicationRequest?_format=json&subject=Patient%2F11f2b925-43b2-45e4-ac34-7811a9eb9c1b
GET https://r4.smarthealthit.org/MedicationDispense?_format=json&subject=Patient%2F11f2b925-43b2-45e4-ac34-7811a9eb9c1b
GET https://r4.smarthealthit.org/MedicationAdministration?_format=json&subject=Patient%2F11f2b925-43b2-45e4-ac34-7811a9eb9c1b
GET https://r4.smarthealthit.org/MedicationStatement?_format=json&subject=Patient%2F11f2b925-43b2-45e4-ac34-7811a9eb9c1b
GET https://r4.smarthealthit.org/Condition/9a459588-d2e2-4f83-8155-327757db91ed?_format=json
GET https://r4.smarthealthit.org/Condition/7446b6f9-7cc2-4692-8f5e-31e0de1b1a86?_format=json


{'resourceType': 'Bundle',
 'id': '592c83b5-ffb6-4cb2-b4ce-0476aa54f091',
 'type': 'collection',
 'timestamp': '2021-09-16T12:46:57Z',
 'entry': [{'fullUrl': 'https://r4.smarthealthit.org/Patient/11f2b925-43b2-45e4-ac34-7811a9eb9c1b',
   'resource': {'resourceType': 'Patient',
    'id': '11f2b925-43b2-45e4-ac34-7811a9eb9c1b',
    'meta': {'versionId': '4',
     'lastUpdated': '2021-04-07T02:56:50.506-04:00',
     'tag': [{'system': 'https://smarthealthit.org/tags',
       'code': 'synthea-5-2019'}]},
    'text': {'status': 'generated',
     'div': '<div xmlns="http://www.w3.org/1999/xhtml">Generated by <a href="https://github.com/synthetichealth/synthea">Synthea</a>.Version identifier: v2.4.0-100-g26a4b936\n .   Person seed: -4773009115611363560  Population seed: 1559319163074</div>'},
    'extension': [{'url': 'http://hl7.org/fhir/us/core/StructureDefinition/us-core-race',
      'extension': [{'url': 'ombCategory',
        'valueCoding': {'system': 'urn:oid:2.16.840.1.113883.6.238',
 

### What should we do about "bad" references?

Some examples of "bad" references
- Invalid reference value – points to a server that doesn't exist
- Server can’t find resource by ID
- Unknown reference format ...

We could
- Raise an error as soon as we hit a bad reference
    - you won't get any data if there is even 1 problem with a reference )o:
- Silently ignore problems and just get what we can
    - you will know that references are missing (o: but won't know why )o:
- Build up a list of issues that can be retured with the patient bundle
    - you can choose what to do about each kind of issue
    - TODO: this is probably the preferred option

In [ ]:
#export
def save_single_patient_medication_bundle(bundle, output_path='data'):
    "Write a patient medication bundle to file."
    Path(output_path).mkdir(exist_ok=True)
    patient = bundle['entry'][0]['resource']
    if patient['resourceType'] != 'Patient':
        raise Exception(f'expected a patient but found {patient}')
    patient_id = patient['id']
    f_name = f'{output_path}/patient_medication_bundle_{patient_id}.json'
    with open(f_name, 'w') as f:
        json.dump(bundle, f, indent=2)
    print('Bundle saved to', f_name)

Now we can save the JSON bundle to file to pass on to the next step of the process (o:

In [ ]:
save_single_patient_medication_bundle(bundle)

Bundle saved to data/patient_medication_bundle_11f2b925-43b2-45e4-ac34-7811a9eb9c1b.json


## Bundle cleanup

The result of `create_single_patient_medication_bundle` is a `collection`, so we need to remove `search` elements from each `entry`. This removes some validation errors reported by https://inferno.healthit.gov/validator/ - thanks Mike (o:

TODO: Do we care what the `search` element is telling us? i.e. what if it's not `match`?

In [ ]:
#export
def handle_entry_search(bundle):
    "Remove `search` elements from each `entry`"
    for entry in bundle['entry']:
        if 'search' in entry: del entry['search']
    return bundle

## Bundle filtering

TODO: Moved to 20a_status_filter - clean this up

### Medication status filter

Remove medication if the status tells us it was not or will not be taken. 

- https://www.hl7.org/fhir/valueset-medicationrequest-status.html
- https://www.hl7.org/fhir/valueset-medicationdispense-status.html
- https://www.hl7.org/fhir/valueset-medication-admin-status.html
- https://www.hl7.org/fhir/valueset-medication-statement-status.html


#### Statuses that we want to remove from the bundle

- MedicationRequest (Include: active, on-hold, completed, entered-in-error, unknown)
    - cancelled 
        - The prescription has been withdrawn before any administrations have occurred
    - stopped 
        - Actions implied by the prescription are to be permanently halted, before all of the administrations occurred. 
        - TODO: This is a ? **halted, before all ...** i.e. might some of the administrations occured
    - draft
        - The prescription is not yet 'actionable'
     
     
- MedicationDispense (Include: on-hold, completed, unknown)
    - preparation	
        - The core event has not started yet, 
    - in-progress
        - The dispensed product is ready for pickup
    - cancelled	
        - The dispensed product was not and will never be picked up by the patient
    - entered-in-error	
        - The dispense was entered in error and therefore nullified
    - stopped	
        - Actions implied by the dispense have been permanently halted, before all of them occurred
        - TODO: This is a ? **hatled, before all ...** i.e. might some of the actions occured
    - declined	
        - The dispense was declined and not performed.
        
- MedicationAdministration (Include: in-progress, on-hold, completed, stopped, unknown)
    - not-done	
        - The administration was terminated prior to any impact on the subject
    - entered-in-error	
        - The administration was entered in error and therefore nullified
        
- MedicationStatement (Include: active, completed, entered-in-error, intended, stopped, on-hold, unknown)
    - not-taken
        - The medication was not consumed by the patient

#### What if these statuses are not appropriate for every study?

It's possible that a study needs to see medication records confirming that a medication was not taken.

i.e. If previous treatment with a medication is an exclusion criteria, absense of a medication record might not be enough to be sure the patient didn't take it.

So we'll need to make filters configurable ...

#### Should we always run the status filter?

The CMOCCUR part of FHIR-to-CDISC Mappings xlsx includes status filtering instructions.

i.e We might not want to implement status filtering on the patient medication bundle.

So we'll need to make filters optional ...

In [ ]:
#export
def medication_status_filter(entry):
    "Remove medications if the status tells us the medication was not or will not be taken"
    statuses_to_remove_map = dict(
        MedicationRequest=['cancelled','stopped','draft'],
        MedicationDispense=['preparation','in-progress','cancelled','entered-in-error','stopped','declined'],
        MedicationAdministration=['not-done','entered-in-error'],
        MedicationStatement=['not-taken'])
    resource = entry.get('resource', {})
    resourceType, status = resource.get('resourceType'), resource.get('status')
    statuses_to_remove = statuses_to_remove_map.get(resourceType)
    if statuses_to_remove is not None and status in statuses_to_remove:
        print('Removing', resourceType, 'with status', status)
        return False
    return True

### "Do Not Perform" filter

In [ ]:
#export
def do_not_perform_filter(entry):
    "Remove medications that have the `doNotPerform` flag set to true"
    resource = entry.get('resource', {})
    if resource.get('doNotPerform', False):
        print('Removing', resource.get('resourceType'), 'with doNotPerform = true')
        return False
    return True

## Create medication bundles for all subjects in a study

When the HAPI FHIR server is available, we should be able to do something like

```
api_base = 'http://hapi.fhir.org/baseR4'
```
Find a patient in a study
```
get_as_raw_json(api_base, 'ResearchSubject')
```
List all resources associated with a study
```
research_study_id = 1171831
# Note: &_revinclude=* gives us everything refering to the study
get_as_raw_json(api_base, 'ResearchStudy', dict(_id=research_study_id, _revinclude='*'))
```
Pick a patient from the above bundle and pull medication requests &darr;
```
# 'subject': {'reference': 'Patient/0c4a1143-8d1c-42ed-b509-eac97d77c9b2'
get_as_raw_json(api_base, 'MedicationRequest', dict(subject='0c4a1143-8d1c-42ed-b509-eac97d77c9b2'))
```
Create medication bundles for all subjects in a study &darr;
```
study_and_subject_bundle = get_as_raw_json(
        api_base, 'ResearchStudy', 
        dict(_id=research_study_id, _revinclude='ResearchSubject:study'))
for i, entry in enumerate(study_and_subject_bundle['entry']):
    resource = entry.get('resource', {})
    if resource.get('resourceType', 'unk') != 'ResearchSubject': continue
    patient_reference = resource.get('individual',{}).get('reference')[8:]
    bundle = create_single_patient_medication_bundle(api_base, patient_reference)
    bundle = handle_entry_search(bundle)
    bundle = filter_bundle(bundle, medication_status_filter)
    bundle = filter_bundle(bundle, do_not_perform_filter)
    save_single_patient_medication_bundle(bundle)
    if i>1: break # stop early (o:
```
Note: &uarr; We're starting to build a bundle processing pipeline (by adding calls to `handle_entry_search` and `filter_bundle`) - and we'll add more functions like this to remove non-concomitant medications etc

## Convert FHIR bundle to SDTM csv

Jay Gustafson built https://mylinks-prod-sdtmtool.azurewebsites.net/TransformBundle that allows parsing a FHIR bundle into SDTM csv content.

Also, you can POST a raw json string to https://mylinks-prod-sdtmtool.azurewebsites.net/TransformBundle/Process and it will return a JSON object containing the SDTM csv content in the following structure:
```
{'cmcsv': '"STUDYID","DOMAIN","USUBJID",...\r\n"RWD-STUDY-01","CM","RWD-SUBJECT-01-30",...\r\n',
 'suppcmcsv': '"STUDYID","RDOMAIN","USUBJID","IDVAR","IDVARVAL","QNAM","QLABEL","QVAL"\r\n"RWD-STUDY-01","CM","RWD-SUBJECT-01-30","CMSEQ","1","CMSOURCE","Resource Name","MedicationRequest"\r\n...',
 'dmcsv': '"STUDYID","DOMAIN","USUBJID",...\r\n"RWD-STUDY-01","DM","RWD-SUBJECT-01-30",...\r\n'}
```

In [ ]:
#demo
response = requests.post('https://mylinks-prod-sdtmtool.azurewebsites.net/TransformBundle/Process', json=bundle)

### View the response as a table

In [ ]:
#demo
pd.read_csv(StringIO(response.json()['cmcsv']))

,STUDYID,DOMAIN,USUBJID,CMSEQ,CMGRPID,CMSPID,CMTRT,CMMODIFY,CMDECOD,CMCAT,...,CMENDTC,CMSTDY,CMENDY,CMDUR,CMSTRF,CMENRF,CMSTRTPT,CMSTTPT,CMENRTPT,CMENTPT
0,RWD-STUDY-01,CM,RWD-SUBJECT-01-30,1,1,SPONSOR-1,24hr nicotine transdermal patch,NaN,NaN,CAT1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,RWD-STUDY-01,CM,RWD-SUBJECT-01-30,2,1,SPONSOR-2,1 ML DOCEtaxel 20 MG/ML Injection,NaN,NaN,CAT1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,RWD-STUDY-01,CM,RWD-SUBJECT-01-30,3,1,SPONSOR-3,Lasix 40mg,NaN,NaN,CAT1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,RWD-STUDY-01,CM,RWD-SUBJECT-01-30,4,1,SPONSOR-4,Galantamine 4 MG Oral Tablet,NaN,NaN,CAT1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,RWD-STUDY-01,CM,RWD-SUBJECT-01-30,5,1,SPONSOR-5,10 ML Furosemide 10 MG/ML Injection,NaN,NaN,CAT1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,RWD-STUDY-01,CM,RWD-SUBJECT-01-30,6,1,SPONSOR-6,0.25 ML Leuprolide Acetate 30 MG/ML Prefilled ...,NaN,NaN,CAT1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,RWD-STUDY-01,CM,RWD-SUBJECT-01-30,7,1,SPONSOR-7,Memantine hydrochloride 2 MG/ML Oral Solution,NaN,NaN,CAT1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,RWD-STUDY-01,CM,RWD-SUBJECT-01-30,8,1,SPONSOR-8,Leucovorin 100 MG Injection,NaN,NaN,CAT1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,RWD-STUDY-01,CM,RWD-SUBJECT-01-30,9,1,SPONSOR-9,10 ML oxaliplatin 5 MG/ML Injection,NaN,NaN,CAT1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,RWD-STUDY-01,CM,RWD-SUBJECT-01-30,10,1,SPONSOR-10,24 HR metoprolol succinate 100 MG Extended Rel...,NaN,NaN,CAT1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
#hide
from nbdev.export import notebook2script
notebook2script()

Converted 00_core.ipynb.
Converted 10_per_patient.ipynb.
Converted 20a_status_filter.ipynb.
Converted 30_cli.ipynb.
Converted 50_web_app.ipynb.
Converted 50a_web_demo.ipynb.
Converted index.ipynb.
